# test data

## test data 전처리

### Part 1: 디스크립터 계산 & with_desc 저장

In [ ]:
# ===================== Part 1: 디스크립터 계산 & with_desc 저장 =====================
import os
import numpy as np
import pandas as pd

# 단일 입력/출력 경로
DATA_FILE = " "
OUT_DIR   = " "
os.makedirs(OUT_DIR, exist_ok=True)

# RDKit 2D descriptor 유틸
from rdkit import Chem
from rdkit.Chem import Descriptors
try:
    from rdkit.Chem import Descriptors3D
    desc3d = {n for n, _ in Descriptors3D._descList}
except Exception:
    desc3d = set()

DESC_2D_NAMES = [n for n, _ in Descriptors._descList if n not in desc3d]
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
_calc = MolecularDescriptorCalculator(DESC_2D_NAMES)

def rdkit_2d_descriptors_from_series(smiles_series: pd.Series,
                                     keep_all_rows: bool = True) -> pd.DataFrame:
    rows, idxs = [], []
    for idx, smi in smiles_series.items():
        smi = "" if pd.isna(smi) else str(smi).strip()
        if not smi:
            if keep_all_rows:
                rows.append([np.nan] * len(DESC_2D_NAMES)); idxs.append(idx)
            continue
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            if keep_all_rows:
                rows.append([np.nan] * len(DESC_2D_NAMES)); idxs.append(idx)
            continue
        try:
            vals = list(_calc.CalcDescriptors(mol))
        except Exception:
            vals = [np.nan] * len(DESC_2D_NAMES)
        rows.append(vals); idxs.append(idx)
    return pd.DataFrame(rows, columns=DESC_2D_NAMES, index=idxs)

def add_rdkit_2d_descriptors(df: pd.DataFrame,
                             smiles_col: str = "SMILES",
                             keep_all_rows: bool = False) -> pd.DataFrame:
    desc_df = rdkit_2d_descriptors_from_series(df[smiles_col], keep_all_rows=keep_all_rows)
    # keep_all_rows=False → RDKit 계산 성공한 행만 남도록 inner-join 효과
    return df.join(desc_df, how="inner") if not keep_all_rows else pd.concat([df, desc_df.reindex(df.index)], axis=1)

# --- 메인  ---
assay_name = os.path.splitext(os.path.basename(DATA_FILE))[0]
print(f"\n[INFO][P1] Processing single file: {DATA_FILE}")

# 1) 로드
df = pd.read_csv(DATA_FILE, dtype=str, engine="python")

# 2) 필수 컬럼 확인 (SMILES, Sample ID만 필요)
required = {"SMILES", "Sample ID"}
if not required.issubset(df.columns):
    raise ValueError(f"Input must have SMILES and Sample ID columns. Found: {list(df.columns)}")

# 3) 문자열 정리 + 빈 SMILES 제거(로그용)
df["SMILES"] = df["SMILES"].astype(str).str.strip()
df["Sample ID"] = df["Sample ID"].astype(str).str.strip()
before = len(df)
df = df[df["SMILES"].str.len() > 0]
removed_empty = before - len(df)
if removed_empty > 0:
    print(f"  -> Removed {removed_empty} rows with empty SMILES.")

# 4) 디스크립터 계산 (유효 SMILES만)
df_with_desc = add_rdkit_2d_descriptors(df, smiles_col="SMILES", keep_all_rows=False)

# 5) 저장: 정제 없이 그대로 저장
out_path = os.path.join(OUT_DIR, f"{assay_name}_with_desc.csv")
df_with_desc.to_csv(out_path, index=False)

print(f"[READY][P1] saved -> {out_path} (rows={len(df_with_desc)}, cols={len(df_with_desc.columns)})")



[INFO][P1] Processing single file: /home/ssm-user/LAIDD/tox21/Data/test_data/test_data.csv


[02:44:05] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 10
[02:44:08] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 11 12 13 14


[READY][P1] saved -> /home/ssm-user/LAIDD/tox21/Data/test_data/test_data_with_desc.csv (rows=645, cols=219)


### Part 2: _with_desc.csv 전처리 후 _2Ddesc.csv 저장

In [ ]:
# ====== Part 2 (TEST용, 단일 파일): _with_desc.csv 전처리 후 _2Ddesc.csv 저장

import os
import numpy as np
import pandas as pd

# --- 입력/출력 경로 ---
OUT_DIR         = " "
WITH_DESC_FILE  = os.path.join(OUT_DIR, "test_data_with_desc.csv")
SAVE_BASE       = os.path.join(OUT_DIR, "test_data_processed")
os.makedirs(OUT_DIR, exist_ok=True)

# --- 공통 NaN 컬럼 드롭 리스트 ---
DROP_LIST_CSV   = " __/columns_with_nan_in_all_12_assays.csv"

def load_common_nan_columns_from_csv(csv_path: str) -> set[str]:
    cols = set()
    if not os.path.exists(csv_path):
        print(f"[P2-Test] Drop-list CSV not found: {csv_path} (skip)")
        return cols
    try:
        df = pd.read_csv(csv_path)
        if "column" in df.columns:
            cols.update(df["column"].dropna().astype(str).str.strip().tolist())
        else:
            first_col = df.columns[0]
            cols.update(df[first_col].dropna().astype(str).str.strip().tolist())
    except Exception as e:
        print(f"[WARN] Failed to read CSV drop list: {csv_path} ({e})")
    return cols

print(f"\n[INFO][P2-Test] Load with_desc: {WITH_DESC_FILE}")
df = pd.read_csv(WITH_DESC_FILE, dtype=str, engine="python")

required = {"SMILES", "Sample ID"}
if not required.issubset(df.columns):
    raise ValueError(f"with_desc must have SMILES and Sample ID. Found: {list(df.columns)}")

df["SMILES"]    = df["SMILES"].astype(str).str.strip()
df["Sample ID"] = df["Sample ID"].astype(str).str.strip()

COMMON_NAN_COLS = load_common_nan_columns_from_csv(DROP_LIST_CSV)

non_feat  = {"Sample ID", "SMILES"}
feat_cols = [c for c in df.columns if c not in non_feat]
if COMMON_NAN_COLS:
    drop_cols_common = sorted(set(feat_cols).intersection(COMMON_NAN_COLS))
    if drop_cols_common:
        print(f"  -> Drop common-NaN columns first: {len(drop_cols_common)}")
        df = df.drop(columns=drop_cols_common, errors="ignore")
        feat_cols = [c for c in feat_cols if c not in drop_cols_common]
    else:
        print("  -> No common-NaN columns to drop in this file.")
else:
    print("  -> No common-NaN drop list loaded or list is empty.")

# 숫자화 후 float64로 통일
X = df[feat_cols].apply(pd.to_numeric, errors="coerce").astype(np.float64)

# NaN 포함 열 전부 삭제
nan_cols = X.columns[X.isna().any(axis=0)].tolist()
if nan_cols:
    print(f"  -> Drop columns containing NaN ({len(nan_cols)}): "
          f"{nan_cols[:5]}{'...' if len(nan_cols) > 5 else ''}")
    X = X.drop(columns=nan_cols)
else:
    print("  -> No columns contain NaN.")

if X.shape[1] == 0:
    raise RuntimeError("No feature columns remain after dropping NaN columns. Check preprocessing settings.")

# 비유한값 최종 점검 (NaN/±inf 모두 없어야 함)
assert np.isfinite(X.values).all(), "Processed X still has non-finite values (NaN or ±inf)."

left_part = df.loc[:, ["Sample ID", "SMILES"]].reset_index(drop=True)
X_final   = X.reset_index(drop=True)
df_save   = pd.concat([left_part, X_final], axis=1)
df_save.to_csv(SAVE_BASE + ".csv", index=False)

print(f"[READY][P2-Test] saved -> {SAVE_BASE+'.csv'} "
      f"(rows={len(df_save)}, cols={len(df_save.columns)}) | "
      f"NaN columns dropped={len(nan_cols)}")



[INFO][P2-Test] Load with_desc: /home/ssm-user/LAIDD/tox21/Data/test_data/test_data_with_desc.csv
  -> Drop common-NaN columns first: 12
  -> No columns contain NaN.
[READY][P2-Test] saved -> /home/ssm-user/LAIDD/tox21/Data/test_data/test_data_processed.csv (rows=645, cols=207) | NaN columns dropped=0


In [ ]:
X

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,14.030275,14.030275,0.039440,-1.257037,0.276843,31.018868,726.919,672.487,726.410483,284.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.407017,11.407017,0.000000,-4.437336,0.262347,35.625000,430.087,420.007,428.944976,124.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.707162,12.707162,0.046209,0.046209,0.540057,13.272727,287.322,274.218,287.105862,106.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13.168842,13.168842,0.000000,-0.356126,0.608509,10.680000,364.823,345.671,364.115397,132.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12.458058,12.458058,0.128272,-3.780944,0.779959,12.782609,351.855,333.711,351.069592,122.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,8.017361,8.017361,0.267361,0.267361,0.469276,8.400000,92.163,84.099,92.029586,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
641,8.873357,8.873357,0.138819,0.138819,0.468960,21.000000,219.285,198.117,219.158292,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
642,4.644676,4.644676,0.472222,0.472222,0.480265,9.285714,116.149,112.117,116.015667,38.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
643,11.178917,11.178917,0.264441,-1.134614,0.646146,19.190476,344.267,329.147,343.031288,112.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Part 3: test

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import joblib

# ===================== 경로 설정 =====================
MODEL_DIR   = "  "
MODEL_PATH  = os.path.join(MODEL_DIR, "multilabel_rf_best.joblib")
ASSAYS_PATH = os.path.join(MODEL_DIR, "assays.json")

TEST_PATH   = " "  # 이미 전처리 완료본
TEST_OUT    = os.path.join(MODEL_DIR, "2D_predictions.csv")    # 스태킹용 출력

# ===================== 유틸 =====================
def load_assays(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"assays.json not found: {path}")
    with open(path, "r") as f:
        assays = json.load(f)
    if not isinstance(assays, list) or not assays:
        raise ValueError("assays.json is not a non-empty list.")
    return assays

def get_training_feature_names(pipe) -> list:
    """
    학습 시 사용된 피처명(순서 포함)을 파이프라인에서 복구.
    - 1순위: 첫 스텝(SimpleImputer 등)의 feature_names_in_
    - 2순위: 파이프라인 자체의 feature_names_in_
    """
    if hasattr(pipe, "named_steps") and "imp" in pipe.named_steps:
        imp = pipe.named_steps["imp"]
        if hasattr(imp, "feature_names_in_"):
            return list(imp.feature_names_in_)
    if hasattr(pipe, "feature_names_in_"):
        return list(pipe.feature_names_in_)
    raise RuntimeError(
        "Cannot find training feature names in the fitted pipeline. "
        "Ensure the model was trained with a pandas DataFrame so steps expose 'feature_names_in_'."
    )

# ===================== 메인 =====================
if __name__ == "__main__":
    print("\n[TEST] Loading model & assays ...")
    pipe = joblib.load(MODEL_PATH)       # Pipeline(SimpleImputer + MultiOutput(RandomForest))
    assays = load_assays(ASSAYS_PATH)
    expected_features = get_training_feature_names(pipe)
    print(f"[TEST] #features expected by model: {len(expected_features)}")

    # (선택) 내부 차원 로그
    try:
        n_expected_internal = pipe.named_steps["clf"].estimators_[0].n_features_in_
        print(f"[TEST] Model internal n_features_in_: {n_expected_internal}")
    except Exception:
        pass

    print(f"[TEST] Loading test data: {TEST_PATH}")
    test_df = pd.read_csv(TEST_PATH, low_memory=False,
                          dtype={"Sample ID":"string","SMILES":"string"})
    # 키 정리
    test_df["Sample ID"] = test_df["Sample ID"].astype("string").str.strip()
    test_df["SMILES"]    = test_df["SMILES"].astype("string").str.strip()

    # 피처 부분만 분리
    non_feat = {"Sample ID", "SMILES", "toxicity"}
    test_feat_cols = [c for c in test_df.columns if c not in non_feat]
    X_test = test_df[test_feat_cols].copy()

    # 수치 변환 + 안전장치(±inf -> NaN)
    X_test = X_test.apply(pd.to_numeric, errors="coerce")
    X_test = X_test.replace([np.inf, -np.inf], np.nan).astype("float64")

    # 학습 스키마로 정렬: 누락 컬럼은 자동으로 NaN 생성, 여분 컬럼은 드롭
    exp_set = set(expected_features)
    cur_set = set(X_test.columns)
    missing = sorted(exp_set - cur_set)
    extra   = sorted(cur_set - exp_set)

    if missing:
        print(f"[TEST][WARN] Missing {len(missing)} feature(s) in test "
              f"(filled as NaN). e.g., {missing[:5]}{'...' if len(missing)>5 else ''}")
    if extra:
        print(f"[TEST] Dropping {len(extra)} unexpected test feature(s). "
              f"e.g., {extra[:5]}{'...' if len(extra)>5 else ''}")

    X_test = X_test.reindex(columns=expected_features)

    assert list(X_test.columns) == list(expected_features)  # 학습 피처와 완전 동일

    # ===== 예측 =====
    print("[TEST] Predicting probabilities ...")
    proba_out = pipe.predict_proba(X_test)  # MultiOutputClassifier: list of (n,2) or ndarray (n,2) when single
    proba_list = proba_out if isinstance(proba_out, list) else [proba_out]
    prob_mat = np.column_stack([p[:, 1] for p in proba_list]).astype(np.float32)

    # ===== 저장: Sample ID, SMILES + assay별 확률 =====
    out_df = pd.concat(
        [test_df.loc[:, ["Sample ID", "SMILES"]].reset_index(drop=True),
         pd.DataFrame(prob_mat, columns=assays)],
        axis=1
    )
    os.makedirs(os.path.dirname(TEST_OUT), exist_ok=True)
    out_df.to_csv(TEST_OUT, index=False)
    print(f"[READY] Saved stacking-ready test probabilities -> {TEST_OUT}")
    print(f"        rows={len(out_df)}, assays={len(assays)}")



[TEST] Loading model & assays ...
[TEST] #features expected by model: 205
[TEST] Model internal n_features_in_: 205
[TEST] Loading test data: /home/ssm-user/LAIDD/tox21/Data/test_data/test_data_processed.csv
[TEST] Predicting probabilities ...
[READY] Saved stacking-ready test probabilities -> /home/ssm-user/LAIDD/tox21/Results_imputer_inf/2D_predictions.csv
        rows=645, assays=12
